In [7]:
import pandas as pd
import numpy as np
import spacy
import re

In [2]:
nlp = spacy.load("trained_spacy_model")

In [4]:
df = pd.read_csv("view_2078.csv")

In [33]:
from sklearn.model_selection import train_test_split
df_sample = df.sample(3000)

In [27]:
def get_matches(text, pattern):
    if pattern == "CAP":
        # Define a pattern to match whole words; assume "CAP" is a placeholder for a word.
        pattern = r'\bCAP\b'
    else:
        # Ensure that pattern matches whole words
        pattern = r'\b' + pattern + r'\w*'
    # Perform the match with case insensitivity
    matches = re.finditer(pattern, text, re.IGNORECASE)
    return [(match.start(), match.end(), match.group()) for match in matches]



In [28]:
def annotate_entities(text, pattern, label):
    matches = get_matches(text, pattern)
    entities = []
    labeled_texts = []
    for start, end, matched_text in matches:
        entities.append((start, end, label))
        labeled_texts.append((start, end, matched_text))
    return entities, labeled_texts


In [34]:
df_sample['Complaint/s'] = df_sample['Complaint/s'].fillna('')
df_sample['History of Illness'] = df_sample['History of Illness'].fillna('')
df_sample['Preliminary Diagnosis/Final Diagnosis'] = df_sample['Preliminary Diagnosis/Final Diagnosis'].fillna('')
df_sample['text'] = df_sample['Complaint/s'] + ' ' + df_sample['History of Illness'] + ' ' + df_sample['Preliminary Diagnosis/Final Diagnosis']

In [35]:
df_sample['text'] = df_sample['text'].apply(lambda x: x.replace('\n', ' '))
df_sample['text'] = df_sample['text'].apply(lambda x: x.replace('\r', ' '))
df_sample['text'] = df_sample['text'].apply(lambda x: x.replace("(+)", ''))
df_sample['text'] = df_sample['text'].apply(lambda x: x.replace("(-)", ''))

In [36]:
df_sample['text']

4958    BP spikes 1 day PTC, patient has dizziness. BO...
3818    follow up occ cough no nasal catarrh, no dob o...
3405    general check up Patient came in for general c...
7149                eye redness  Bacterial conjunctivitis
7333    hypogastric pain 3 days PTC patient had hypoga...
                              ...                        
3999    hypogastric pain decreased urinary frequency 1...
3515    Headache Runny nose 3 days PTC - runny nose, h...
6539    cough 1 week PTC, patient had cough and itchy ...
4072    Fever 3 days PTC patient had fever associated ...
1358    “Ring Worm” Patient complained of having ring ...
Name: text, Length: 3000, dtype: object

In [ ]:
symptoms = ['fever', 'cough', 'colds']
diseases = ['URTI', 'Upper Respiratory Tract Infection', 'CAP', 'Community Acquired Pneumonia', 'PCAP']
ILI = symptoms + diseases
id = 0
jsonl = []
for disease in ILI:
    for text in df_sample['text']:
        entities, labeled_texts = annotate_entities(text, disease, disease)
        jsonl.append((id, {'entities': entities, 'text': text}))
        id += 1

In [56]:
import re
import pandas as pd
import json

def get_matches(text, pattern):
    # Create a regex pattern to match whole words
    if pattern == "CAP":
        # Define a pattern to match whole words; assume "CAP" is a placeholder for a word.
        pattern = r'\bCAP\b'
    else:
        # Ensure that pattern matches whole words
        pattern = r'\b' + pattern + r'\w*' 
    # Perform the match with case insensitivity
    matches = re.finditer(pattern, text, re.IGNORECASE)
    return [(match.start(), match.end(), match.group()) for match in matches]

def annotate_entities(text, entity_id_counter):
    diseases = [
        'fever', 'cough', 'colds', 'URTI', 'Upper Respiratory Tract Infection', 
        'CAP', 'Community Acquired Pneumonia', 'PCAP', 'Asthma', 'Bronchitis', 
        'Pharyngitis', 'Laryngitis', 'Sinusitis', 'Pneumonia', 'Chronic Cough'
    ]
    entities = []
    for disease in diseases:
        matches = get_matches(text, disease)
        for start, end, _ in matches:
            entities.append({
                'id': entity_id_counter[0],  # Unique ID for each entity
                'label': 'ILI',
                'start_offset': start,
                'end_offset': end
            })
            entity_id_counter[0] += 1
    return entities

entry_id_counter = 1  # Counter for entry IDs
entity_id_counter = [1]  # Mutable list to maintain the global counter for entities
jsonl = []

jsonl_file_path = 'output.jsonl'

with open(jsonl_file_path, 'w') as jsonl_file:
    for text in df_sample['text']:
        entities = annotate_entities(text, entity_id_counter)
        entry = {
            'id': entry_id_counter,  # Unique ID for each entry
            'text': text,
            'tag': np.random.randint(0,2),  # Assuming a fixed value for 'tag'; adjust as necessary
            'entities': entities,
            'relations': [],  # Empty list for relations
            'Comments': []  # Empty list for comments
        }
        jsonl_file.write(json.dumps(entry) + '\n')
        entry_id_counter += 1  # Increment entry ID after each entry

print(f"Data saved to {jsonl_file_path}")

Data saved to output.jsonl


In [55]:
np.random.randint(0,2, size = 10)

array([0, 1, 1, 0, 1, 0, 0, 1, 1, 1])